In [1]:
from top import read_trees, tree_level
from utils import Statistics, merge_statistics

In [2]:
gold_trees = read_trees("/efs/dengcai/top/out/gold.parse")
tot = len(gold_trees)

def read_scores(fname):
    parse_scores = {}
    mx = 0
    for l in open(fname).readlines():
        if l.startswith("H-"):
            l = l.strip()
            l = l.split("\t")
            h_num = int(l[0].split('-')[1])
            h_score = float(l[1].strip())
            assert h_num not in parse_scores
            parse_scores[h_num] = h_score
            mx = max(h_num, mx)
    assert len(parse_scores) == mx + 1
    return [ parse_scores[i] for i in range(mx+1)]


def compute(old_trees, pred_trees, pred_scores):
    assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
    final_pred_trees = []
    for i in range(tot):
        pred_trees_i = pred_trees[i*beam:i*beam+beam]
        pred_scores_i = pred_scores[i*beam:i*beam+beam]
        idx = list(range(beam))
        idx.sort(key=lambda x:-pred_scores_i[x])
        picked = idx[0]
        final_pred_trees.append(pred_trees_i[picked])
    stats = tree_level(gold_trees, old_trees, final_pred_trees)
    stat = merge_statistics(stats)
    return stat

In [3]:
# Table 1: compute the mean and standard deviation.
array = []
for old in '123 456 789 555 666'.split():
    old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")
    pred_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")
    stats = tree_level(gold_trees, old_trees, pred_trees)
    stat = merge_statistics(stats)
    EM, sEM = stat["whole"].new_acc, stat["atom"].new_acc
    array.append([EM, sEM])
import numpy as np
data = np.array(array)
for avg, std in zip(data.mean(0), data.std(0)):
    print (f"{avg*100:.2f} {std*100:.2f}")

array = []
for old in '123 456 789 555 666'.split():
    old_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")
    pred_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")
    stats = tree_level(gold_trees, old_trees, pred_trees)
    stat = merge_statistics(stats)
    EM, sEM = stat["whole"].new_acc, stat["atom"].new_acc
    array.append([EM, sEM])
import numpy as np
data = np.array(array)
for avg, std in zip(data.mean(0), data.std(0)):
    print (f"{avg*100:.2f} {std*100:.2f}")


85.67 0.10
90.34 0.10
86.97 0.11
91.11 0.12


In [4]:
# Table 6: compute the upper-bounds of BCR 
for NUM in [8, 10]:
    for dropout in [0.25, 0.3]:
        all_nfi_stats = []
        all_acc_stats = []
        for old in "123 456 789 555 666".split():
            for new in "1 2 3 4 5".split():
                print (f"{old}->{new}")
                effective_nfi_stats = []
                effective_acc_stats = []
                old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

                pred_trees, pred_scores = [], []
                for sample_seed in range(1, NUM+1):
                    pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/base{new}.dropout{dropout}sample{sample_seed}.full.parse")

                for i in range(tot):
                    gold_trees_i = [ gold_trees[i] for t in range(NUM) ]
                    old_trees_i = [ old_trees[i] for t in range(NUM) ]
                    pred_trees_i = pred_trees[i::tot]

                    stats = tree_level(gold_trees_i, old_trees_i, pred_trees_i)
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:stats[x]["atom"].nfr if "atom" in stats[x].keys else 1000000 )
                    picked = idx[0]
                    effective_nfi_stats.append(stats[picked])
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:-stats[x]["atom"].new_acc if "atom" in stats[x].keys else 1000000)
                    picked = idx[0]
                    effective_acc_stats.append(stats[picked])
                stat = merge_statistics(effective_nfi_stats)
                all_nfi_stats.append(stat)
                stat = merge_statistics(effective_acc_stats)
                all_acc_stats.append(stat)
        print (dropout)
        print ("nfr")
        stat = merge_statistics(all_nfi_stats)
        stat.report(["whole", "atom"])
        print ("acc")
        stat = merge_statistics(all_acc_stats)
        stat.report(["whole", "atom"])
        
for NUM in [8, 10]:
    for dropout in [0.25, 0.3]:
        all_nfi_stats = []
        all_acc_stats = []
        for old in "123 456 789 555 666".split():
            for new in "1 2 3 4 5".split():
                print (f"{old}->{new}")
                effective_nfi_stats = []
                effective_acc_stats = []
                old_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")

                pred_trees, pred_scores = [], []
                for sample_seed in range(1, NUM+1):
                    pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse")

                for i in range(tot):
                    gold_trees_i = [ gold_trees[i] for t in range(NUM) ]
                    old_trees_i = [ old_trees[i] for t in range(NUM) ]
                    pred_trees_i = pred_trees[i::tot]

                    stats = tree_level(gold_trees_i, old_trees_i, pred_trees_i)
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:stats[x]["atom"].nfr if "atom" in stats[x].keys else 1000000 )
                    picked = idx[0]
                    effective_nfi_stats.append(stats[picked])
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:-stats[x]["atom"].new_acc if "atom" in stats[x].keys else 1000000)
                    picked = idx[0]
                    effective_acc_stats.append(stats[picked])
                stat = merge_statistics(effective_nfi_stats)
                all_nfi_stats.append(stat)
                stat = merge_statistics(effective_acc_stats)
                all_acc_stats.append(stat)
        print (dropout)
        print ("nfr")
        stat = merge_statistics(all_nfi_stats)
        stat.report(["whole", "atom"])
        print ("acc")
        stat = merge_statistics(all_acc_stats)
        stat.report(["whole", "atom"])
        
for NUM in [10]:
    for dropout in [0.3]:
        all_nfi_stats = []
        all_acc_stats = []
        for old in "123 456 789 555 666".split():
            for new in "1 2 3 4 5".split():
                print (f"{old}->{new}")
                effective_nfi_stats = []
                effective_acc_stats = []
                old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

                pred_trees, pred_scores = [], []
                for sample_seed in range(1, NUM+1):
                    pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse")

                for i in range(tot):
                    gold_trees_i = [ gold_trees[i] for t in range(NUM) ]
                    old_trees_i = [ old_trees[i] for t in range(NUM) ]
                    pred_trees_i = pred_trees[i::tot]

                    stats = tree_level(gold_trees_i, old_trees_i, pred_trees_i)
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:stats[x]["atom"].nfr if "atom" in stats[x].keys else 1000000 )
                    picked = idx[0]
                    effective_nfi_stats.append(stats[picked])
                    idx = list(range(NUM))
                    idx.sort(key=lambda x:-stats[x]["atom"].new_acc if "atom" in stats[x].keys else 1000000)
                    picked = idx[0]
                    effective_acc_stats.append(stats[picked])
                stat = merge_statistics(effective_nfi_stats)
                all_nfi_stats.append(stat)
                stat = merge_statistics(effective_acc_stats)
                all_acc_stats.append(stat)
        print (dropout)
        print ("nfr")
        stat = merge_statistics(all_nfi_stats)
        stat.report(["whole", "atom"])
        print ("acc")
        stat = merge_statistics(all_acc_stats)
        stat.report(["whole", "atom"])

In [ ]:
# Table 5: BCR (beam search)
for beam in [2, 4, 8, 10]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}=>{new} {beam}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/base{new}.b{beam}.full.parse")
            pred_scores = read_scores(f"/efs/dengcai/top/out/base{new}.b{beam}.full.parse.base{old}.score")
            stat = compute(old_trees, pred_trees, pred_scores)
            all_stats.append(stat)
    stat = merge_statistics(all_stats)
    print ("="*55)
    stat.report(["whole", "atom"])
    print ("="*55)
for beam in [2, 4, 8, 10]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}=>{new} {beam}")
            old_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/large{new}.b{beam}.full.parse")
            pred_scores = read_scores(f"/efs/dengcai/top/out/large{new}.b{beam}.full.parse.large{old}.score")
            stat = compute(old_trees, pred_trees, pred_scores)
            all_stats.append(stat)
    stat = merge_statistics(all_stats)
    print ("="*55)
    stat.report(["whole", "atom"])
    print ("="*55)
    
for beam in [2, 4, 8, 10]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}=>{new} {beam}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/large{new}.b{beam}.full.parse")
            pred_scores = read_scores(f"/efs/dengcai/top/out/large{new}.b{beam}.full.parse.base{old}.score")
            stat = compute(old_trees, pred_trees, pred_scores)
            all_stats.append(stat)
    stat = merge_statistics(all_stats)
    print ("="*55)
    stat.report(["whole", "atom"])
    print ("="*55)

In [ ]:
# Table 3 & 5 : BCR (dropout-p sampling)
for dropout in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (dropout)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


In [ ]:
# Table 3: distillation
m1 = "base"
m2 = "base"
all_stats = []

for s1 in "123 456 789 555 666".split():
    old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
    pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}.{m1}{s1}.seed1.parse")
    stats = tree_level(gold_trees, old_trees, pred_trees)
    stat = merge_statistics(stats)
    all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

m1 = "large"
m2 = "large"
all_stats = []

for s1 in "123 456 789 555 666".split():
    old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
    pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}.{m1}{s1}.seed1.parse")
    stats = tree_level(gold_trees, old_trees, pred_trees)
    stat = merge_statistics(stats)
    all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

m1 = "base"
m2 = "large"
all_stats = []

for s1 in "123 456 789 555 666".split():
    old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
    pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}.{m1}{s1}.seed1.parse")
    stats = tree_level(gold_trees, old_trees, pred_trees)
    stat = merge_statistics(stats)
    all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

In [ ]:
# Table 3: untreated
m1 = "base"
m2 = "base"
all_stats = []

for s1 in "123 456 789 555 666".split():
    for s2 in "1 2 3 4 5".split():
        old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
        pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}{s2}.parse")
        stats = tree_level(gold_trees, old_trees, pred_trees)
        stat = merge_statistics(stats)
        all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

m1 = "large"
m2 = "large"
all_stats = []

for s1 in "123 456 789 555 666".split():
    for s2 in "1 2 3 4 5".split():
        old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
        pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}{s2}.parse")
        stats = tree_level(gold_trees, old_trees, pred_trees)
        stat = merge_statistics(stats)
        all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

m1 = "base"
m2 = "large"
all_stats = []

for s1 in "123 456 789 555 666".split():
    for s2 in "1 2 3 4 5".split():
        old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
        pred_trees = read_trees(f"/efs/dengcai/top/out/{m2}{s2}.parse")
        stats = tree_level(gold_trees, old_trees, pred_trees)
        stat = merge_statistics(stats)
        all_stats.append(stat)
stat = merge_statistics(all_stats)
stat.report(["whole", "atom"])

In [ ]:
# Table 3 & Figure 3: ensemble
m1 = "base"
m2 = "base"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "123 456 789 555 666".split():
        for s2 in "A B C D E".split():
            s2 = f"{ensemble_size}{s2}"
            old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

m1 = "large"
m2 = "large"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "123 456 789 555 666".split():
        for s2 in "A B C D E".split():
            s2 = f"{ensemble_size}{s2}"
            old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

m1 = "base"
m2 = "large"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "123 456 789 555 666".split():
        for s2 in "A B C D E".split():
            s2 = f"{ensemble_size}{s2}"
            old_trees = read_trees(f"/efs/dengcai/top/out/{m1}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])   

In [ ]:
# Table 5, top-k sampling
for topk in [5, 10, 50, 100]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/base{new}.topk{topk}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/base{new}.topk{topk}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topk)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])
        
        
for topk in [5, 10, 50, 100]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.large{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topk)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

for topk in [5, 10, 50, 100]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topk)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

In [ ]:
# Table 5, top-p sampling
for topp in [0.95, 0.9, 0.85, 0.8]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/base{new}.topp{topp}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/base{new}.topp{topp}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topp)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])
    
for topp in [0.95, 0.9, 0.85, 0.8]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/large{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.large{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topp)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


for topp in [0.95, 0.9, 0.85, 0.8]:
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, 11):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(10))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topp)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


In [ ]:
# Figure 2:  the number of candidates
dropout = 0.35
for num in range(10, 11):
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, num+1):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.dropout{dropout}sample{sample_seed}.full.parse.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(num))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (dropout)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

topp = 0.95
for num in range(1, 11):
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, num+1):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topp{topp}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(num))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topp)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


topk=5
for num in range(1, 11):
    all_stats = []
    for old in "123 456 789 555 666".split():
        for new in "1 2 3 4 5".split():
            print (f"{old}->{new}")
            old_trees = read_trees(f"/efs/dengcai/top/out/base{old}.parse")

            pred_trees, pred_scores = [], []
            for sample_seed in range(1, num+1):
                pred_trees = pred_trees + read_trees(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.parse")
                pred_scores = pred_scores + read_scores(f"/efs/dengcai/top/out/large{new}.topk{topk}.sample{sample_seed}.base{old}.score")
            assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
            final_pred_trees = []
            for i in range(tot):
                pred_trees_i = pred_trees[i::tot]
                pred_scores_i = pred_scores[i::tot]
                idx = list(range(num))
                assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
                idx.sort(key=lambda x:-pred_scores_i[x])
                picked = idx[0]
                final_pred_trees.append(pred_trees_i[picked])
            stats = tree_level(gold_trees, old_trees, final_pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print (topk)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])

In [ ]:
# ensemble => ensemble
m1 = "base"
m2 = "base"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "A B C D E".split():
        for s2 in "A B C D E".split():
            if s1 == s2:
                continue
            old_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m1}.{ensemble_size}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{ensemble_size}{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


m1 = "large"
m2 = "large"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "A B C D E".split():
        for s2 in "A B C D E".split():
            if s1 == s2:
                continue
            old_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m1}.{ensemble_size}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{ensemble_size}{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])


m1 = "base"
m2 = "large"
for ensemble_size in range(2, 11):
    all_stats = []
    for s1 in "A B C D E".split():
        for s2 in "A B C D E".split():
            if s1 == s2:
                continue
            old_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m1}.{ensemble_size}{s1}.parse")
            pred_trees = read_trees(f"/efs/dengcai/top/out/seq2seq.{m2}.{ensemble_size}{s2}.parse")
            stats = tree_level(gold_trees, old_trees, pred_trees)
            stat = merge_statistics(stats)
            all_stats.append(stat)
    print(ensemble_size)
    stat = merge_statistics(all_stats)
    stat.report(["whole", "atom"])